In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import glob
import os

In [2]:
df_list = []
os_list = os.listdir()
os_list.sort()

csv_list = [file for file in os_list if file.endswith('.csv')]
json_list = [file for file in os_list if file.endswith('.json')]

for i,val in enumerate(csv_list):
    
    # open json and create dict from file
    with open(json_list[i]) as json_data:
        data = json.load(json_data)
    
    df_json = pd.DataFrame(data['selectedAttributes'])
    
    lga_name_to_title = dict(zip(df_json['name'],df_json['title']))
    
    
    # open csv file and clean
    df = pd.read_csv(csv_list[i], index_col=None, header=0)
    
    clean_cols = []

    for col in df.columns:
        clean_cols.append(lga_name_to_title[col.strip(' ')].split(' Age')[0].replace('Persons ','').replace('Females ','').replace('Males ','').replace('emales ','').replace('ales ',''))
    
    df.columns = clean_cols
    
    df = df.groupby(df.columns, axis=1).sum()
    
    col_list = df.columns

    for col in col_list:
        if (('Total' in col) and ('LGA' not in col)):
            df=df.drop(col,axis=1)
            
    df_list.append(df)

In [3]:
df = pd.concat(df_list,axis=1)

In [4]:
df = df.groupby(df.columns, axis=1).sum()

In [5]:
df = df.rename(columns = {'China excludes SARs and Taiwan':'China', 
                         'Korea Republic of South':'South Korea',
                         'South Eastern Europe nfd':'South Eastern Europe',
                         'The Former Yugoslav Republic of Macedonia':'North Macedonia',
                         'Hong Kong SAR of China':'Hong Kong',
                         'LGA Code 2016':'LGA Code',
                         'LGA Name 2016':'LGA Name'})

In [6]:
df = df.drop(columns = ['Born elsewhere','Country of birth not stated'])

In [7]:
df

,Afghanistan,Australia,Bangladesh,Bosnia and Herzegovina,Cambodia,Canada,Chile,China,Croatia,Egypt,...,South Eastern Europe,Sri Lanka,Taiwan,Thailand,North Macedonia,Turkey,United States of America,Vietnam,Wales,Zimbabwe
0,34,83170,170,0,10,80,17,238,89,28,...,62,200,3,125,0,0,191,65,78,58
1,0,46805,108,0,0,99,15,558,0,0,...,0,120,80,43,0,0,310,93,67,45
2,0,67975,3,0,12,163,58,108,19,12,...,8,41,9,124,0,9,282,43,101,58
3,0,3744,0,0,0,0,0,0,0,0,...,0,0,8,12,0,0,0,0,0,0
4,36,69062,65,3,12,108,21,240,43,36,...,28,106,0,93,5,0,120,114,43,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,148,89904,567,47,124,363,47,1902,31,72,...,16,272,144,167,3,13,163,481,77,221
541,0,17429,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
542,0,13136,0,0,3,11,0,54,3,0,...,0,31,7,17,0,0,51,40,11,35
543,14,33715,0,0,124,46,8,59,9,0,...,12,14,14,113,4,0,70,341,31,115


In [8]:
first_col = df.pop('LGA Code')
second_col = df.pop('LGA Name')

df.insert(0, 'LGA Code', first_col)
df.insert(0, 'LGA Name', second_col)

In [9]:
for i,name in enumerate(df['LGA Name']):
    df['LGA Name'][i] = df['LGA Name'][i].split(' (')[0]


<ipython-input-9-aa46115f6696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LGA Name'][i] = df['LGA Name'][i].split(' (')[0]


In [18]:
df.to_csv('clean_country_of_birth_by_lga.csv',index=False)